# Welcome to the Wonderland Vaccine Registration Management Application!
To begin, first run the cell below and enter the database's username and password to connect to the database.

In [ ]:
import mysql.connector
import datetime

def gen_n_spaces(n):
    rv = ""
    for i in range(0, n):
        rv += " "
    return rv

def max_col_len(result, column_names, index):
    cur_max = 0
    for i in range(0, len(result)):
        if (len(str(result[i][index]))) > cur_max:
            cur_max = len(str(result[i][index]))
    if cur_max > len(column_names[index]):
        return cur_max
    else:
        return len(column_names[index])

def print_query_result(result, column_names):
    if len(result) == 0:
        out_str = ""
        for i in range(0, len(column_names)):
            fld = column_names[i]
            out_str += fld + gen_n_spaces(2)
        print(out_str)
        dashes = ""
        for i in range(0, len(out_str)):
            dashes += "-"
        print(dashes)
        print("No results.")
        return;
    for i in range(0, len(result[0]) - len(column_names)):
        column_names.append("")
    out_str = ""
    for i in range(0, len(result[0])):
        fld = column_names[i]
        out_str += fld + gen_n_spaces(max_col_len(result, column_names, i) - len(fld) + 2)
    print(out_str)
    dashes = ""
    for i in range(0, len(out_str)):
        dashes += "-"
    print(dashes)
    for i in range(0, len(result)):
        out_str = ""
        for j in range(0, len(result[0])):
            fld = str(result[i][j])
            out_str += fld + gen_n_spaces(max_col_len(result, column_names, j) - len(fld) + 2)
        print(out_str)

currentDatetime = datetime.datetime(2021, 12, 15, hour = 13, minute = 0, second = 0)

config = {
   'user': input("Username: "),
   'password': input("Password: "),
   'host': 'localhost',
   'port': 3306,
   'database': 'wlvaxinfo2',
   'raise_on_warnings': True                    
}

cnx = mysql.connector.connect(**config)

cur = cnx.cursor()

Run the next cell to start a session as a user of this application from wonderland. For the data in the database after freshly running 'loadAll.sql', odd numbered PINs are health staff and a PIN equal to 1 is the cheif. Run this cell again to start a session as a different user.

In [ ]:
userPIN = input("Enter PIN: ")
staffmode = False
cheifmode = False

if userPIN.isnumeric() and len(userPIN) <= 16:
    query = ("""SELECT first_name, last_name
                FROM folk
                WHERE PIN = """ + userPIN)
    cur.execute(query)
    result = cur.fetchall()
    if len(result) == 1:
        print("Welcome back, " + result[0][0] + " " + result[0][1] + "!")
        query = ("""SELECT is_cheif
                    FROM health_staff
                    WHERE PIN = """ + userPIN)
        cur.execute(query)
        result = cur.fetchall()
        if len(result) == 1:
            staffmode = True
            if result[0][0] == "Yes":
                cheifmode = True
                print("You are using this app in cheif mode.")
            else:
                print("You are using this app in staff mode.")
        else:
            print("You are using this app in user mode.")
    else:
        print("No one in the system has that PIN. Run the cell again.")
else:
    print("PIN must be a positive integer with 16 (or less) digits. Run the cell again.")

The cell below allows an application tester to simulate time passing. Enter the date as YYYY then MM then DD and the time as HH then MM then SS. Hitting enter for each will use the default datetime of "2021-12-15 13:00:00" (1PM).

In [ ]:
currentDatetime = datetime.datetime(2021, 12, 15, hour = 13, minute = 0, second = 0)

year = input("Year: ")
month = input("Month: ")
day = input("Day: ")
h = input("Hour: ")
m = input("Minute: ")
s = input("Second: ")
if len(year+month+day+h+m+s) > 0:
    currentDatetime = datetime.datetime(int(year), int(month), int(day), hour = int(h), minute = int(m), second = int(s))
    
print("\nCurrent Datetime: " + str(currentDatetime))

# Transactions

The cell below allows the cheif to dispatch health staff for a certain date. The current time must be between 10am and 3pm (15:00:00). Note that they are already dispatched for the default date, so dispatching them further isn't necesary, although can be done.

In [ ]:
def check_if_assigned(tup, qinfo):
    for i in range(0, len(qinfo)):
        if tup[0] == str(qinfo[i][0]):
            if tup[1] == str(qinfo[i][1]):
                return True
    return False
    

if cheifmode:
    
    if currentDatetime.time().hour >= 10 and currentDatetime.time().hour < 15:

        query = ("""SELECT PIN
                    FROM health_staff""")
        cur.execute(query)
        hsPINs = cur.fetchall()

        print("These are the health worker's PINs:")
        pin_info = ""
        for i in range(0, len(hsPINs)):
            pin_info += str(hsPINs[i][0])
            if i < len(hsPINs) - 1:
                pin_info += ", "
        print(pin_info)

        query = ("""SELECT place_id
                    FROM health_center""")
        cur.execute(query)
        hcIDs = cur.fetchall()

        print("\nThese are the health center's place_ids:")
        hc_info = ""
        for i in range(0, len(hcIDs)):
            hc_info += str(hcIDs[i][0])
            if i < len(hcIDs) - 1:
                hc_info += ", "
        print(hc_info)

        tomorrow = str(currentDatetime.date() + datetime.timedelta(days=1))
        query = ("""SELECT PIN, place_id
                    FROM dispatched_to
                    WHERE date = \"""" + tomorrow + "\"")
        cur.execute(query)
        dsptmts = cur.fetchall()
        print("\nThese are the current dispatchments")
        print("PIN" + gen_n_spaces(14) + "place_id")
        for i in range(0, len(dsptmts)):
            print(str(dsptmts[i][0]) + gen_n_spaces(17 - len(str(dsptmts[i][0]))) + str(dsptmts[i][1]))

        print("\nEnter the PIN of the health staff you're dispatching.")
        dispPIN = input()
        print("Enter the place_id of the health center you're dispatching them to.")
        dispplaceid = input()
        if not check_if_assigned((dispPIN, dispplaceid), dsptmts):
            transaction = ("""INSERT INTO dispatched_to
                            VALUES (""" + dispPIN + "," + dispplaceid + ",\"" + tomorrow + "\")")
            cur.execute(transaction)
            print("\nDispatchment successful.")
        else:
            print("\nThis dispatchment was already made!")
    
    else:
        print("Dispatchments can't be done at this time!")
    
else:
    print("You are not the chief!")

The cell below allows a health staff member to input the information for a new health center.

In [ ]:
if staffmode:
    place_id = input("place_id: ")
    street_number = input("Street Number: ")
    street_name = input("Street Name: ")
    city = input("City: ")
    state = input("State: ")
    zipcode = input("Zipcode (5 digits)): ")
    coord_x = input("X Coordinate: ")
    coord_y = input("Y Coordinate: ")
    mullinstr = "INSERT INTO place VALUES "
    mullinstr += "(""" + place_id + "," + street_number + ",\""
    mullinstr += street_name + "\",\"" + city + "\",\"" + state + "\","
    mullinstr += zipcode  + "," + coord_x + "," + coord_y + ")"
    transaction = (mullinstr)
    cur.execute(transaction)
    transaction = ("""INSERT INTO health_center
                    VALUES (""" + place_id + ")")
    cur.execute(transaction)
    print("\nHealth center creation successful!")
else:
    print("You are not a member of staff!")

The cell below allows a staff member to change a folk's PIN.

In [ ]:
if staffmode:
    query = ("""SELECT PIN, first_name, last_name
                FROM folk""")
    cur.execute(query)
    folkdata = cur.fetchall()

    print("\nThese are the current PINs and associated people:")
    for i in range(0, len(folkdata)):
        print(str(folkdata[i][0]) + " " + folkdata[i][1] + " " + folkdata[i][2])
    
    oldPIN = input("\nEnter the PIN you want to change: ")
    newPIN = input("New value: ")
    transaction = ("UPDATE folk SET PIN = " + newPIN + " WHERE PIN = " + oldPIN)
    cur.execute(transaction)
    if (oldPIN == userPIN):
        userPIN = newPIN
        
    print("\nPin change successful!")
    
else:
    print("You are not a member of staff!")

The cell below allows any user to register for a vaccine appointment or change their existing registration. Starting a session as user 7-12 and choosing the date "2022-01-07" will have the most edge-case-y results for the number of people already registered.

In [ ]:
print("Enter the date you want to register for/change your registration to.")
year = input("Year: ")
month = input("Month: ")
day = input("Day: ")

reg_date = datetime.date(int(year), int(month), int(day))

if reg_date <= currentDatetime.date():
    print("\nIt's too late to register for that date!")
else:
    query = ("""SELECT local_centers.place_id, COUNT(PIN)
    FROM (SELECT place_id
    FROM health_center NATURAL JOIN place
    WHERE city = (SELECT city
            FROM folk JOIN place
            ON folk.res_id = place.place_id
            WHERE PIN = """+userPIN+""")) AS local_centers
        LEFT OUTER JOIN
        (SELECT PIN, place_id
        FROM place NATURAL JOIN
        health_center NATURAL JOIN registered_for
        WHERE date = \""""+str(reg_date)+"""\") as res_info
        ON local_centers.place_id = res_info.place_id
    GROUP BY place_id""")
    cur.execute(query)
    hcdata = cur.fetchall()

    mullinstr = "\nThese are the health centers in your city's place_ids "
    mullinstr += "along with the current number of people registered for that date:\n"
    print(mullinstr)
    for i in range(0, len(hcdata)):
        print(str(hcdata[i][0]) + ": " + str(hcdata[i][1]) + " people registered.")

    print("\nEnter the place_id of the health center you want to register for.")
    hcpi = input()

    transaction = ("INSERT INTO registered_for VALUES ("+userPIN+","+hcpi+",\""+str(reg_date)+"""\")
                    ON DUPLICATE KEY
                    UPDATE
                    place_id = """+hcpi+", date = \""+str(reg_date)+"\"")
    cur.execute(transaction)

    print("\nRegistration successful!")

The below cell allows a staff member to delete an existing health center.

In [ ]:
if staffmode:
    query = ("""SELECT place_id
                FROM health_center""")
    cur.execute(query)
    hcIDs = cur.fetchall()
    
    print("\nThese are the health center's place_ids:")
    hc_info = ""
    for i in range(0, len(hcIDs)):
        hc_info += str(hcIDs[i][0])
        if i < len(hcIDs) - 1:
            hc_info += ", "
    print(hc_info)
    
    place_id = input("\nEnter the place_id of the health_center to be deleted: ")
    transaction = ("""DELETE FROM place
                    WHERE place_id IN (SELECT place_id FROM health_center)
                    AND place_id = """ + place_id)
    cur.execute(transaction)
    print("\nHealth center deletion successful!")
else:
    print("You are not a member of staff!")

The cell below allows a staff member to update the coordinates of a place.

In [ ]:
if staffmode:
    query = ("""SELECT place_id, coord_x, coord_y
                FROM place""")
    cur.execute(query)
    hcinfo = cur.fetchall()
    
    print("\nThese are the current place_ids along with their current coordinates:")
    for i in range(0, len(hcinfo)):
        print(str(hcinfo[i][0]) + ": (" + str(hcinfo[i][1]) + ", " + str(hcinfo[i][2]) + ")")
    
    place_id = input("\nEnter the place_id of the place you wish to change the coordinates of: ")
    coord_x = input("Enter the new X coordinate: ")
    coord_y = input("Enter the new Y coordinate: ")
    transaction = ("""UPDATE place
                    SET coord_x = """ + coord_x + ", coord_y = " + coord_y + """
                    WHERE place_id = """ + place_id)
    cur.execute(transaction)
    print("\nCoordinate change successful!")
else:
    print("You are not a member of staff!")

# Queries/Reports

The cell below will list the name and age of all folks.

In [ ]:
print("Name and age of all folks:\n")

query = ("""SELECT CONCAT(first_name, " ", last_name) AS name, 
                CASE
                    WHEN
                        MONTH(today) < MONTH(dob)
                        OR (MONTH(today) = MONTH(dob)
                        AND DAY(today) < DAY(dob))
                    THEN
                        YEAR(today) - YEAR(dob) - 1
                    ELSE
                        YEAR(today) - YEAR(dob)
                END AS age
            FROM folk, (SELECT \""""+str(currentDatetime.date())+"\" AS today) AS temp")
cur.execute(query)
result = cur.fetchall()
print_query_result(result, ["Name", "Age"])

The cell below will list the city name, state, and number of residents for each city.

In [ ]:
print("City, state, and number of residents of each city:\n")

query = ("""SELECT city, state, COUNT(res_id) as resident_count
            FROM (residence NATURAL JOIN place)
                JOIN folk ON residence.place_id = folk.res_id
            GROUP BY city
            ORDER BY resident_count DESC""")
cur.execute(query)
result = cur.fetchall()
print_query_result(result, ["City", "State", "Number of Residents"])

The cell below will list state names and the number of places in each state.

In [ ]:
print("States and their number of places:\n")

query = ("""SELECT state, count(residence.place_id)
            FROM place LEFT OUTER JOIN residence ON place.place_id = residence.place_id
            GROUP BY state
            ORDER BY state ASC;""")
cur.execute(query)
result = cur.fetchall()
print_query_result(result, ["State", "Number of Places"])

The cell below will list the PINs of all the people registered for a given health center between two given dates. After a fresh run of loadAll.sql, all folks will be registered between December 2021 and February 2022.

In [ ]:
query = ("""SELECT place_id
            FROM health_center""")
cur.execute(query)
hcIDs = cur.fetchall()

print("\nThese are the health center's place_ids:")
hc_info = ""
for i in range(0, len(hcIDs)):
    hc_info += str(hcIDs[i][0])
    if i < len(hcIDs) - 1:
        hc_info += ", "
print(hc_info)

print("\nEnter the place_id of the health center you want information about.")
hcid = input()
print("Enter the beginning date of the range.")
year1 = input("Year: ")
month1 = input("Month: ")
day1 = input("Day: ")
begin_date = datetime.date(int(year1), int(month1), int(day1))
print("Enter the end date of the range.")
year2 = input("Year: ")
month2 = input("Month: ")
day2 = input("Day: ")
end_date = datetime.date(int(year2), int(month2), int(day2))

print("\nPINs of the users who are registered there and then:\n")

query = ("""SELECT PIN
            FROM registered_for
            WHERE date BETWEEN \""""+str(begin_date)+"\" AND \""+str(end_date)+"""\"
            AND place_id = """ + hcid)
cur.execute(query)
result = cur.fetchall()
print_query_result(result, ["PIN", "Number of Places"])

The cell below will give the number of staff working at health centers within 5 miles of the center of Megapolis in a given month, with the exception of a list of locations provided by the user.

In [ ]:
month = input("Month: ")

mullinstr = "\nEnter in a list of coordinates of locations to exclude, "
mullinstr += "hit return at the beginning of an entry to finish.\n"
print(mullinstr)
coords = []
done = False
while not done:
    coord_x = input("X: ")
    if len(coord_x) != 0:
        coord_y= input("Y: ")
        coords.append((coord_x, coord_y))
    else:
        done = True
        
print()
        
query = """SELECT COUNT(DISTINCT PIN) as staff_within_five_miles
        FROM dispatched_to NATURAL JOIN health_center NATURAL JOIN place
        WHERE POWER(coord_x, 2) + POWER(coord_y, 2) <= 25 """
if (len(coords) > 0):
    query += "AND (coord_x, coord_y) NOT IN ("
    for i in range(0, len(coords)):
        query += "(" + coords[i][0] + "," + coords[i][1] + ")"
        if i < len(coords) - 1:
            query += ","
    query += ") "
query += "AND MONTH(date) = " + month
cur.execute(query)
result = cur.fetchall()
print_query_result(result, ["Number Of Staff"])

The cell below will find the most popular health centers for a given month within a given city. The default cities are "Megapolis" and "The Primary Keys".

In [ ]:
month = input("Month: ")
city = input("City: ")

print("\nMost popular health centers in that city in that month:\n")

query = ("""SELECT place.*
            FROM place NATURAL JOIN health_center NATURAL JOIN registered_for
                JOIN
                (SELECT city, MONTH(date) as month, MAX(popularity) AS most_pop
                FROM place NATURAL JOIN health_center NATURAL JOIN registered_for
                    NATURAL JOIN
                    (SELECT place_id, COUNT(PIN) AS popularity
                    FROM place NATURAL JOIN health_center NATURAL JOIN registered_for
                    GROUP BY city, MONTH(date), place_id) AS pop_info
                GROUP BY city, month) AS max_pop_info ON place.city = max_pop_info.city
                                    AND MONTH(registered_for.date) = max_pop_info.month
            WHERE place.city = \"""" + city + "\" AND MONTH(date) = " + month + """
            GROUP BY place.city, max_pop_info.month, place_id
            HAVING COUNT(PIN) = ROUND(AVG(most_pop), 0)""")
cur.execute(query)
result = cur.fetchall()
print_query_result(result, ["place_id", "Street Number", "Street Name", "City", "State",
                            "Zip Code", "X Coordinate", "Y Coordinate"])

The cell below will find the staff that have been dispatched to every health center in a given state. Default state names are "Oheyo" and "Floorida".

In [ ]:
state = input("State: ")

print("\nStaff that have been dispatched to every health center in that state:\n")

query = ("""SELECT folk.*
            FROM folk NATURAL JOIN
                ((SELECT PIN, state, COUNT(dispatched_to.place_id) as num_dispatched
                FROM dispatched_to NATURAL JOIN health_center NATURAL JOIN place
                GROUP BY PIN, state) AS dispatch_info
                JOIN
                (SELECT state, COUNT(place_id) as num_hcs
                FROM health_center NATURAL JOIN place
                GROUP BY state) as state_hc_info
                ON dispatch_info.state = state_hc_info.state
                    AND dispatch_info.num_dispatched = state_hc_info.num_hcs)
            WHERE dispatch_info.state = \"""" + state +"\"")
cur.execute(query)
result = cur.fetchall()
print_query_result(result, ["PIN", "First Name", "Last Name", "DOB", "Work Phone",
                            "Mobile Phone", "Home Phone", "Residence place_id"])

The cell below finds folks that are registered to a health center further away than the closest one to their residence.

In [ ]:
print("Folks registered to a health center further away than the closest one to them:\n")

query = ("""SELECT folk.*
FROM folk NATURAL JOIN
    (SELECT PIN, POWER(hc_place_info.coord_x - res_place_info.coord_x, 2)
        + POWER(hc_place_info.coord_y - res_place_info.coord_y, 2) as squared_distance
    FROM (place as hc_place_info) NATURAL JOIN health_center NATURAL JOIN
        registered_for NATURAL JOIN folk JOIN (place as res_place_info)
        ON folk.res_id = res_place_info.place_id) AS dist_to_reg
    JOIN
    (SELECT PIN, MIN(POWER(hc_place_info.coord_x - res_place_info.coord_x, 2)
            + POWER(hc_place_info.coord_y - res_place_info.coord_y, 2)) as squared_distance
    FROM folk JOIN (place as res_place_info)
        ON folk.res_id = res_place_info.place_id,
        health_center NATURAL JOIN (place as hc_place_info)
    GROUP BY (PIN)) AS closest_hc
    ON dist_to_reg.PIN = closest_hc.PIN
WHERE dist_to_reg.squared_distance > closest_hc.squared_distance""")
cur.execute(query)
result = cur.fetchall()
print_query_result(result, ["PIN", "First Name", "Last Name", "DOB", "Work Phone",
                            "Mobile Phone", "Home Phone", "Residence place_id"])

The cell below finds the number of people registered for a given month who live within 10 miles of a given location. By default all residences are pretty close to either (0, 0) or (1337, 2021).

In [ ]:
month = input("Month: ")
coord_x = input("X coordinate:")
coord_y = input("X coordinate:")

print("\nNumber of people registered within 10 miles of that location in that month:")

cur.execute("SELECT num_folks_registered("+month+", "+coord_x+", "+coord_y+")")
result = cur.fetchall()
print(result[0][0])

# Exiting

Run the below cell when you're done using the application to close the connection.

In [ ]:
cur.close()
cnx.close()